
# **Project 1: Colour Compression**
Huy G. Tong $^{1 *}$ \
$^{1}$ *Faculty of Information Technology, VNUHCM - University of Science, Vietnam*


### **ABSTRACT**
**K-means clustering** is an unsupervised learning algorithm used to group data points into **$k$** number of clusters. This algorithm is used extensively in machine learning and data science. For this project, we will implement $k$-means clustering to reduce the number of colours of an input image, hence colour compression. The *PDF* file included in this folder has demonstrated the overall introduction to the algorithm as well as step by step instructions. This Notebook will then focus on the implementation aspect of the project.

### **IMPLEMENTATION**
This section will display the implementation of the K-means algorithm in Python. First we have to import all necessary packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL

Next, we need a function to generate `K` initial centroids based on the input mode

In [ ]:
def generate_centroids(data, n_chan, k, mode="random"):
    if mode == 'in_pixels':
        return data[np.random.choice(len(data), size=k, replace=False)]
    
    centroids = np.random.choice(256, (k, n_chan))
    while len(np.unique(centroids, axis=0)) != k:
        centroids = np.random.choice(256, (k, n_chan))

    return centroids

Our K-means function will be installed bellow. This function will perform the main calculation for K-means clustering, the output of this function will be used to reduce the number of colours of the original image to a number of `K` colours.

In [ ]:
def k_means_clustering(img_1d, clusters, max_iter=1000, init_centroids="random", quiet=False):
    '''
    K-Means algorithm
    
    Inputs: 
        img_1d : np.ndarray with shape=(height * width, num_channels)
            Original image in 1d array
       
        clusters : int
            Number of clusters
            
        max_iter : int
            Max iterations, default is 1000
            
        init_centroids : str
            The way which use to init centroids
            'random' --> centroid has `c` channels, with `c` is initial random in [0,255]
            'in_pixels' --> centroid is a random pixels of original image

        quiet : bool
            Option on whether to display progress on the terminal
            
    Outputs:
        centroids : np.ndarray with shape=(k_clusters, num_channels)
            Store color centroids
            
        labels : np.ndarray with shape=(height * width, )
            Store label for pixels (cluster's index on which the pixel belongs)

    '''

    assert init_centroids in ["random", "in_pixels"], \
        "Centroid initialization mode not found"
    
    n_channels = img_1d.shape[-1]
    centroids = generate_centroids(img_1d, n_channels, clusters, mode=init_centroids)

    if not quiet:
        print("Initial centroids:")
        print(centroids)
    
    labels = np.array([0 for _ in range(img_1d.shape[0])])
    
    for i in range(max_iter):
        
        # Calculate the Euclidean distances between each data point and centroids
        #
        #   distance.shape == (K_clusters, num_data_points)
        # 
        # Each row corresponds to a centroid, each column coresponds to a datapoint,
        # and each value is the distance between a data point and a centroid
        distances = np.sqrt(np.sum(np.square(img_1d - centroids[:, np.newaxis]), axis=2))

        # Assign data points to closest centroid by selecting the 
        # index of the minimum distance
        labels = np.argmin(distances, axis=0)
        
        new_centroids = []

        for k in range(clusters):
            members = img_1d[labels == k]

            if members.size != 0:
                cent_mean = members.mean(axis=0)
                new_centroids.append(cent_mean)
            
            # Discard the empty cluster
            else:
                if not quiet:
                    print("Empty cluster encountered!")
                    
                clusters -= 1
            
        new_centroids = np.array(new_centroids)
        
        if not quiet and i % 10 == 0:
            print(f"--- Iteration {str(i)}:\n Current centroids: \n{str(centroids)}\n")
            print(f"New centroids: \n{str(new_centroids)}\n")

        if centroids.shape[0] == new_centroids.shape[0] \
            and np.all(centroids == new_centroids): break

        centroids = new_centroids 

    if not quiet: 
        print("\nFINISHED! Empty clusters are automatically removed from the final result\n")
        print("Final:")
        print(centroids)

    return centroids, labels

The majority of our program's functionalities will be put inside the `compress_colours` function.

In [ ]:
def compress_colours(pixels, image_size, K_clusters, init_mode="random"):
    '''
    Reduce the number of colours in an image to a number of K colours
    
    Inputs:
        pixels : np.ndarray with shape=(width, height, num_channels)
            Original image as a numpy array

        image_size : tuple of int
            Width and height of the image

        K_clusters : int
            Number of expected colour clusters
            
    Outputs:
        img_compressed : np.ndarray with shape=(width, height, num_channels)
            Reconstructed image as a numpy array with no more than K colours

    '''

    width, height = image_size

    # Flatten out the array for easier calculation
    num_channels = pixels.shape[-1]
    pixels = np.reshape(pixels, (-1, num_channels))

    # Main calculations will happen here
    colour_centroids, labels = k_means_clustering(
        img_1d=pixels,
        clusters=K_clusters,
        init_centroids=init_mode
    )

    # Assign the values of the centroids to its member pixels, 
    # which will reduce the amount of colours in the image
    for i_label, labels in enumerate(labels):
        pixels[i_label] = colour_centroids[labels]

    # Reshape the pixels into the original shape
    img_compressed = np.reshape(pixels, (width, height, num_channels))

    return img_compressed

Now, we can start preparing our data for the fun part. Our input would be a `2D image` and each pixel is a combination of multiple colour channels. For example, each pixel of an `RGB` image take 3 values from 0 to 255 to represent the intensity of the *Red*, *Green*, and *Blue* making up the colour of that pixel. A `CMYK` image works similarly, but instead of 3 values per pixel, it is 4 values corresponding to *Cyan*, *Magenta*, *Yellow*, and *Key* which is black.

In [ ]:
def handle_input():
    img_path = input("Enter image path: ")

    K_clusters = input("Enter the number of clusters: ")
    K_clusters = int(K_clusters)

    init_mode = input("Enter the centroid initialization mode (random, in_pixels): ")
    assert init_mode in ["random", "in_pixels"], "Initialization mode not found"

    output_type = input("Enter output file type (pdf, png)")
    assert output_type in ["png", "pdf"], "Output file type not supported"

    with PIL.Image.open(img_path) as img:
        pixels = np.array(img)

    return img_path, pixels, K_clusters, init_mode, output_type


Finally, the `main` function will act as our driver. In this function, we will process the image using the defined function, display the image with `MatPlotLib`, as well as export the compressed image as the user-defined file type:

In [ ]:
def main():

    img_path, pixels, K_clusters, init_mode, output_type = handle_input()

    # Save a copy of the original for presentation purposes
    original = pixels.copy()

    width, height = original.shape[0], original.shape[1]

    img_compressed = compress_colours(
        pixels=pixels, 
        image_size=(width, height),
        K_clusters=K_clusters,
        init_mode=init_mode
    )

    # Save compressed image
    # img_out = PIL.Image.fromarray(img_compressed)
    plt.imsave(img_path + "_k=" + str(K_clusters) + "_compressed." + output_type, img_compressed)
    print(f"Compressed image saved at {img_path}_k={str(K_clusters)}_compressed.{output_type}")

    # Left plot showing the original input image
    plt.subplot(1, 2, 1)
    plt.imshow(original)
    plt.title(f"Original {img_path}")

    # Right plot showing the compressed colours image
    plt.subplot(1, 2, 2)
    plt.imshow(img_compressed)
    plt.title(f"Colours compressed with k = {str(K_clusters)}")

    fig = plt.gcf()
    fig.set_size_inches((15, 11), forward=False)
    fig.savefig(img_path + "_original_k=" + str(K_clusters) + "_comparison." + output_type)

    plt.show()

Let us call the `main` function and observe the results!

In [ ]:
main()

### **REFERENCES**
- ["$k$-means clustering"](https://en.wikipedia.org/wiki/K-means_clustering), **Wikipedia**
- ["K means Clustering – Introduction"](https://www.geeksforgeeks.org/k-means-clustering-introduction/), **GeeksforGeeks**
- ["How to Convert images to NumPy array?"](https://www.geeksforgeeks.org/how-to-convert-images-to-numpy-array/), by *priyarajtt* on **GeeksforGeeks**
- ["Bài 4: K-means Clustering"](https://machinelearningcoban.com/2017/01/01/kmeans/), **Machine Learning cơ bản**
- ["How do I use np.newaxis?"](https://stackoverflow.com/questions/29241056/how-do-i-use-np-newaxis), by *Yue Harriet Huang* on **Stack Overflow**
- ["Numpy Axes, Explained"](https://www.sharpsightlabs.com/blog/numpy-axes-explained/), by *Joshua Ebner* on **Sharp Sight**
- [Numpy documentation on broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)